# EHRMAMBA on MIMIC-IV (Demo)

This notebook trains the EHRMAMBA model (Mamba/SSM-based foundation model for EHR) on a MIMIC-IV in-hospital mortality prediction task. Same pipeline as the Transformer example; only the model class changes.

In [ ]:
from pyhealth.datasets import MIMIC4Dataset, get_dataloader, split_by_sample
from pyhealth.tasks import InHospitalMortalityMIMIC4
from pyhealth.models import EHRMamba
from pyhealth.trainer import Trainer

In [ ]:
# Use the bundled MIMIC-IV demo (test-resources/core/mimic4demo). For full MIMIC-IV,
# download from PhysioNet and set ehr_root to that path (e.g. /app/data/mimic4 in Docker).
from pathlib import Path
_repo = Path.cwd().parent if (Path.cwd().parent / "test-resources").exists() else Path.cwd()
ehr_root = str(_repo / "test-resources" / "core" / "mimic4demo")

dataset = MIMIC4Dataset(
    ehr_root=ehr_root,
    ehr_tables=["diagnoses_icd", "procedures_icd", "prescriptions", "labevents"],
    dev=True,
)

In [ ]:
task = InHospitalMortalityMIMIC4()
sample_dataset = dataset.set_task(task)
train_dataset, val_dataset, test_dataset = split_by_sample(sample_dataset, ratios=[0.7, 0.1, 0.2])

In [ ]:
train_loader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = EHRMamba(
    dataset=sample_dataset,
    embedding_dim=128,
    num_layers=2,
    state_size=16,
    conv_kernel=4,
    dropout=0.1,
)
trainer = Trainer(model=model, metrics=["roc_auc", "pr_auc"])

In [ ]:
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=5,
    monitor="roc_auc",
    optimizer_params={"lr": 1e-4},
)

In [ ]:
trainer.evaluate(test_loader)